In [49]:
import numpy as np
import pandas as pd
import os

In [50]:
np_path="/home/lijia/codes/202211/sage/result/8615-id-42-np"
group_attrs=['Gender', 'Age', 'Race', 'Rosy_Cheeks', 'Shiny_Skin', 'Bangs', 
             'Sideburns', 'HairColor', 'Beard', 'FaceShape', 'Double_Chin', 'High_Cheekbones', 
             'Chubby', 'Forehead_visible', 'Brown_Eyes', 'Bags_Under_Eyes', 'Bushy_Eyebrows', 
             'Arched_Eyebrows', 'Mouth_Closed', 'Smiling', 'Big_Lips', 'NoseType', 
             'Heavy_Makeup', 'Eyeglasses', 'Wearing_Lipstick', 'Attractive']

feature_groups = {'Gender': ['Male','Female'],
 'Age':['Young', 'Middle_Aged', 'Senior'],
 'Race':['Asian','White','Black'],
 'Rosy_Cheeks': ['Rosy_Cheeks'],
 'Shiny_Skin': ['Shiny_Skin'],
 'Bangs':['Bangs'],
 'Sideburns':['Sideburns'],
 'HairColor': ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair'],
 'Beard': ['No_Beard','Mustache','5_o_Clock_Shadow','Goatee'],
 'FaceShape': ['Oval_Face', 'Square_Face', 'Round_Face'],
 'Double_Chin': ['Double_Chin'],
 'High_Cheekbones': ['High_Cheekbones'],
 'Chubby': ['Chubby'],
 'Forehead_visible': ['Obstructed_Forehead','Fully_Visible_Forehead'],
 'Brown_Eyes':['Brown_Eyes'],
 'Bags_Under_Eyes':['Bags_Under_Eyes'],
 'Bushy_Eyebrows':['Bushy_Eyebrows'],
 'Arched_Eyebrows':['Arched_Eyebrows'],
 'Mouth_Closed':['Mouth_Closed'],
 'Smiling':['Smiling'],
 'Big_Lips':['Big_Lips'],
 'NoseType': ['Big_Nose','Pointy_Nose'],
 'Heavy_Makeup': ['Heavy_Makeup'],
 'Eyeglasses': ['Eyeglasses'],
 'Wearing_Lipstick': ['Wearing_Lipstick'],
 'Attractive': ['Attractive']}

target_attributions=['Male','Female','Young', 'Middle_Aged', 'Senior','Asian','White','Black','Rosy_Cheeks',
                'Shiny_Skin','Bangs','Sideburns',
                'Black_Hair','Blond_Hair','Brown_Hair','Gray_Hair','No_Beard','Mustache',
                '5_o_Clock_Shadow','Goatee','Oval_Face','Square_Face','Round_Face','Double_Chin',
                'High_Cheekbones','Chubby','Obstructed_Forehead', 'Fully_Visible_Forehead','Brown_Eyes',
                'Bags_Under_Eyes','Bushy_Eyebrows','Arched_Eyebrows','Mouth_Closed','Smiling',
                'Big_Lips','Big_Nose','Pointy_Nose','Heavy_Makeup',
                'Wearing_Lipstick','Eyeglasses','Attractive']

In [51]:
test_pre=pd.read_csv("/home/lsf/桌面/lijia/face-recognition/data/test_result_41.csv")[["Filename","0"]]
test_gt=pd.read_csv("/exdata/data/vggface2/test_id_sample.csv")
maad_file=pd.read_csv("/exdata/data/vggface2/maad_id.csv").drop(["id"],axis=1)
pre_result=test_gt.merge(test_pre,on="Filename")
pre_result["pre_result"]=pre_result["id"]==pre_result["0"]
pre_result=pre_result.drop(["id","0"],axis=1)
pre_result=pre_result.merge(maad_file,on="Filename")[["Filename","pre_result"]+target_attributions]


In [52]:
# 计算属性影响力
names=os.listdir(np_path)
scores_list=[]
for name in names:
    scores_list.append(np.load(os.path.join(np_path,name)))
scores_np=np.array(scores_list)
scores_np_abs=np.abs(scores_np)
mean_score=scores_np_abs.mean(axis=0)
sorted_id=sorted(range(len(mean_score)),key=lambda k:mean_score[k],reverse=True)
concept_dict={}
for id in sorted_id:
    concept_dict[group_attrs[id]]=mean_score[id]
print(concept_dict)

{'HairColor': 0.8424888499115728, 'Beard': 0.601511337725717, 'Race': 0.5818597741083427, 'Forehead_visible': 0.5529543395910371, 'Age': 0.5505088136032509, 'NoseType': 0.4494370128317275, 'FaceShape': 0.4221940637482368, 'Gender': 0.3419755719269145, 'Bushy_Eyebrows': 0.28418640341010815, 'Brown_Eyes': 0.2841340966120544, 'Big_Lips': 0.26599301942389025, 'Arched_Eyebrows': 0.2628840374137106, 'Eyeglasses': 0.26230918429337885, 'Smiling': 0.2584343546704193, 'Shiny_Skin': 0.25704147270986044, 'Bangs': 0.24711719271632315, 'Mouth_Closed': 0.24696221405329985, 'Bags_Under_Eyes': 0.2368655501408108, 'High_Cheekbones': 0.2251373676560469, 'Sideburns': 0.22237071461772337, 'Chubby': 0.21112268135574275, 'Double_Chin': 0.2056447816548898, 'Attractive': 0.18083358355216764, 'Wearing_Lipstick': 0.15212123989929174, 'Heavy_Makeup': 0.14811179183004072, 'Rosy_Cheeks': 0.09125087237070967}


In [59]:
plt_result_plus={}
plt_result_min={}
for n,s in concept_dict.items():
    sub_attrs=feature_groups[n]
    sub_len=len(sub_attrs)
    
    for attr in sub_attrs:
        attr_count=pre_result[pre_result[attr]==1][attr].sum()
        attr_acc=pre_result[pre_result[attr]==1]['pre_result'].sum()
        plt_result_plus[attr]=attr_acc/attr_count
        
    other=pre_result
    for attr in sub_attrs:
        other=other[pre_result[attr]==0]
    other_count=other['pre_result'].sum()
    if(other.shape[0]==0):
        continue
    plt_result_min["no_"+n]=other_count/other.shape[0]

print(plt_result_plus)
print(plt_result_min)
        
    

/home/lijia/anaconda3/envs/sagefc/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/lijia/anaconda3/envs/sagefc/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/lijia/anaconda3/envs/sagefc/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/lijia/anaconda3/envs/sagefc/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/lijia/anaconda3/envs/sagefc/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/lijia/anaconda3/envs/sagefc/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
 

{'Black_Hair': 0.81930825093341, 'Blond_Hair': 0.8259823576583801, 'Brown_Hair': 0.8120026269025729, 'Gray_Hair': 0.9000490287240698, 'No_Beard': 0.8116818203455991, 'Mustache': 0.8358585858585859, '5_o_Clock_Shadow': 0.8534126630454419, 'Goatee': 0.9037585421412301, 'Asian': 0.760430231997299, 'White': 0.8401373747963238, 'Black': 0.8193916349809885, 'Obstructed_Forehead': 0.7283008730691739, 'Fully_Visible_Forehead': 0.8545364688439521, 'Young': 0.8153152586441886, 'Middle_Aged': 0.8749346517498394, 'Senior': 0.8908814214617694, 'Big_Nose': 0.8491700417602249, 'Pointy_Nose': 0.8350632233020958, 'Oval_Face': 0.8294788019358148, 'Square_Face': 0.8505926484402037, 'Round_Face': 0.6213592233009708, 'Male': 0.8377228251776935, 'Female': 0.8142121347120292, 'Bushy_Eyebrows': 0.8594208238154889, 'Brown_Eyes': 0.8080912275592016, 'Big_Lips': 0.8140403661926147, 'Arched_Eyebrows': 0.8411976462443752, 'Eyeglasses': 0.7895613111973235, 'Smiling': 0.8670911094103498, 'Shiny_Skin': 0.843347930116

In [60]:
ori_plus={}
ori_min={}
total=pre_result.shape[0]
for n,s in concept_dict.items():
    sub_attrs=feature_groups[n]
    sub_len=len(sub_attrs)
    
    for attr in sub_attrs:
        attr_count=pre_result[pre_result[attr]==1][attr].sum()
        # attr_acc=pre_result[pre_result[attr]==1]['pre_result'].sum()
        plt_result_plus[attr]=attr_acc/attr_count
        
    other=pre_result
    for attr in sub_attrs:
        other=other[pre_result[attr]==0]
    other_count=other['pre_result'].sum()
    if(other.shape[0]==0):
        continue
    plt_result_min["no_"+n]=other_count/other.shape[0]

print(plt_result_plus)
print(plt_result_min)

43


NameError: name 'abc' is not defined

绘制图像

In [56]:
import matplotlib.pyplot as plt

SyntaxError: unexpected EOF while parsing (<ipython-input-54-6d42f23011db>, line 2)